In [1]:
from sqlalchemy import create_engine, Column, Integer, String
from sqlalchemy.ext.declarative import declarative_base
Base = declarative_base()

In [2]:
# Define our pet table
class Pet(Base):
    __tablename__ = 'pet'
    id = Column(Integer, primary_key=True)
    name = Column(String)
    type = Column(String)
    age = Column(Integer)

In [7]:
# Right now, this table only exists in python and not in the actual database
Base.metadata.tables

immutabledict({'pet': Table('pet', MetaData(bind=None), Column('id', Integer(), table=<pet>, primary_key=True, nullable=False), Column('name', String(), table=<pet>), Column('type', String(), table=<pet>), Column('age', Integer(), table=<pet>), schema=None)})

In [8]:
# Create our database engine
engine = create_engine('sqlite:///pets.sqlite')

In [5]:
# This is where we create our tables in the database
Base.metadata.create_all(engine)

In [6]:
# The ORM’s “handle” to the database is the Session.
from sqlalchemy.orm import Session
session = Session(engine)

## Create Data

In [32]:
# Note that adding to the session does not update the table. It queues up those queries.
session.add(Pet(name='Justin Timbersnake', type='snek', age=2))
session.add(Pet(name='Pawtrick Stewart', type='good boy', age=10))
session.add(Pet(name='Godzilla', type='iguana', age=1))
session.add(Pet(name='Marshmallow', type='polar bear', age=4))

In [33]:
# The data hasn't been added yet
engine.execute('select * from pet').fetchall()

[(1, 'Justin Timbersnake', 'snek', 2),
 (2, 'Pawtrick Stewart', 'good boy', 10),
 (3, 'Godzilla', 'iguana', 1),
 (4, 'Justin Timbersnake', 'snek', 2),
 (5, 'Pawtrick Stewart', 'good boy', 10),
 (6, 'Godzilla', 'iguana', 1),
 (7, 'Marshmallow', 'polar bear', 24),
 (8, 'Justin Timbersnake', 'snek', 2),
 (9, 'Pawtrick Stewart', 'good boy', 10),
 (10, 'Godzilla', 'iguana', 1),
 (11, 'Marshmallow', 'polar bear', 4)]

In [34]:
# We can use the new attribute to see the queue of data ready to go into the database
session.new

IdentitySet([<__main__.Pet object at 0x111bafb70>, <__main__.Pet object at 0x111bafb38>, <__main__.Pet object at 0x1118f55f8>, <__main__.Pet object at 0x1111a8748>])

In [35]:
# commit() flushes whatever remaining changes remain to the database, and commits the transaction.
session.commit()




InvalidRequestError: This Session's transaction has been rolled back due to a previous exception during flush. To begin a new transaction with this Session, first issue Session.rollback(). Original exception was: (sqlite3.IntegrityError) UNIQUE constraint failed: pet.id [SQL: 'UPDATE pet SET id=? WHERE pet.id = ?'] [parameters: (11, 7)] (Background on this error at: http://sqlalche.me/e/gkpj)

In [36]:


# Nothing new to add
session.new

IdentitySet([<__main__.Pet object at 0x111bafb70>, <__main__.Pet object at 0x111bafb38>, <__main__.Pet object at 0x1118f55f8>, <__main__.Pet object at 0x1111a8748>])

In [37]:
# query the database
session.query(Pet.name, Pet.type, Pet.age).all()

InvalidRequestError: This Session's transaction has been rolled back due to a previous exception during flush. To begin a new transaction with this Session, first issue Session.rollback(). Original exception was: (sqlite3.IntegrityError) UNIQUE constraint failed: pet.id [SQL: 'UPDATE pet SET id=? WHERE pet.id = ?'] [parameters: (11, 7)] (Background on this error at: http://sqlalche.me/e/gkpj)

## Update Data

In [29]:
# Create a query and then run update on it
pet = session.query(Pet).filter_by(name="Marshmallow").first()
pet.__dict__

InvalidRequestError: This Session's transaction has been rolled back due to a previous exception during flush. To begin a new transaction with this Session, first issue Session.rollback(). Original exception was: (sqlite3.IntegrityError) UNIQUE constraint failed: pet.id [SQL: 'UPDATE pet SET id=? WHERE pet.id = ?'] [parameters: (11, 7)] (Background on this error at: http://sqlalche.me/e/gkpj)

In [30]:
pet.age += 20

InvalidRequestError: This Session's transaction has been rolled back due to a previous exception during flush. To begin a new transaction with this Session, first issue Session.rollback(). Original exception was: (sqlite3.IntegrityError) UNIQUE constraint failed: pet.id [SQL: 'UPDATE pet SET id=? WHERE pet.id = ?'] [parameters: (11, 7)] (Background on this error at: http://sqlalche.me/e/gkpj)

In [31]:
pet.__dict__

{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState at 0x11194abe0>}

In [22]:
# For modifications, we can use the dirty attribute
pet.id = 11
session.dirty

IdentitySet([<__main__.Pet object at 0x11194aba8>])

In [23]:
# Commit Transaction
session.commit()

IntegrityError: (sqlite3.IntegrityError) UNIQUE constraint failed: pet.id [SQL: 'UPDATE pet SET id=? WHERE pet.id = ?'] [parameters: (11, 7)] (Background on this error at: http://sqlalche.me/e/gkpj)

In [20]:
# Session is up-to-date
session.dirty

IdentitySet([])

In [21]:
session.query(Pet.id, Pet.name, Pet.type, Pet.age).all()

[(1, 'Justin Timbersnake', 'snek', 2),
 (2, 'Pawtrick Stewart', 'good boy', 10),
 (3, 'Godzilla', 'iguana', 1),
 (4, 'Justin Timbersnake', 'snek', 2),
 (5, 'Pawtrick Stewart', 'good boy', 10),
 (6, 'Godzilla', 'iguana', 1),
 (7, 'Marshmallow', 'polar bear', 24),
 (8, 'Justin Timbersnake', 'snek', 2),
 (9, 'Pawtrick Stewart', 'good boy', 10),
 (10, 'Godzilla', 'iguana', 1),
 (11, 'Marshmallow', 'polar bear', 4)]

## Delete Data

In [ ]:
# Create a query and then delete the row collected
pet = session.query(Pet).filter_by(id=4).delete()
session.commit()

In [24]:
session.query(Pet.id, Pet.name, Pet.type, Pet.age).all()

InvalidRequestError: This Session's transaction has been rolled back due to a previous exception during flush. To begin a new transaction with this Session, first issue Session.rollback(). Original exception was: (sqlite3.IntegrityError) UNIQUE constraint failed: pet.id [SQL: 'UPDATE pet SET id=? WHERE pet.id = ?'] [parameters: (11, 7)] (Background on this error at: http://sqlalche.me/e/gkpj)